In [1]:
import argparse
import numpy as np
import pickle
import types
from qiskit import Aer, BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import ExactEigensolver, VQE
from qiskit.aqua.components.optimizers import SPSA, COBYLA, L_BFGS_B
from qiskit.aqua.components.variational_forms import RY, RYRZ
#from qiskit.aqua.operators import Z2Symmetries
from qiskit.chemistry import FermionicOperator
#from qiskit.chemistry.core import Hamiltonian, QubitMappingType, TransformationType
from qiskit.chemistry.drivers import PySCFDriver, UnitsType

args = types.SimpleNamespace()
#args.algorithm='VQE' 
args.basis_set='ccpvdz' 
#args.basis_set='sto3g' 
args.max_parallel_threads=10 
args.molecule='H2' 
args.num_shots=1000
args.outpath_mol='/home/ubuntu/mol2qpu/output/operators/molecule/'
args.outpath_ferm='/home/ubuntu/mol2qpu/output/operators/fermionic/'
args.outpath_qub='/home/ubuntu/mol2qpu/output/operators/qubit/'
args.outpath_vqe='/home/ubuntu/mol2qpu/output/VQE_results/'
args.qubitmapping_type='bravyi_kitaev'
#args.qubitmapping_type='jordan_wigner'
#args.qubitmapping_type='parity'
#args.random_seed=750 
#args.two_qubit_reduce = True
#args.vqe_aer = True
args.vqe_depth=1 
args.vqe_entangler='linear' 
#args.vqe_max_iter=2
#args.vqe_opt_params = False
#args.vqe_optimizer='SPSA' 
#args.vqe_sim = True
#args.vqe_var_form='RY' 

In [30]:
## load qmolecule
d_string = '0pt55'
filename_mol = args.outpath_mol + args.molecule + '_' + args.basis_set + '_' + d_string + '_MOLE' + '.pkl'
qmolecule = pickle.load(open(filename_mol,'rb'))

## load operators
filename_ferm = args.outpath_ferm + args.molecule + '_' + args.basis_set + '_' + d_string + '_FERM' + '.pkl'
filename_qub = args.outpath_qub + args.molecule + '_' + args.basis_set + '_' + d_string + '_QUBIT_bk' + '.pkl'
ferOp = pickle.load(open(filename_ferm, 'rb'))
qubitOp = pickle.load(open(filename_qub, 'rb'))

In [31]:
print("molecule file = ", filename_mol)
print("qubit file = ", filename_qub)
num_particles = qmolecule.num_alpha + qmolecule.num_beta
print("num orbitals = ", qmolecule.num_orbitals)
print("num_particles = ", num_particles)
print("num qubits = ", qubitOp.num_qubits)

molecule file =  /home/ubuntu/mol2qpu/output/operators/molecule/H2_ccpvdz_0pt55_MOLE.pkl
qubit file =  /home/ubuntu/mol2qpu/output/operators/qubit/H2_ccpvdz_0pt55_QUBIT_bk.pkl
num orbitals =  10
num_particles =  2
num qubits =  20


In [41]:
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(circuit_caching=True, 
                                   backend=backend,
                                   backend_options={'max_parallel_threads': args.max_parallel_threads,                                                                             'max_parallel_experiments': 0, 
                                                    'shots': args.num_shots})
optimizer = SPSA(max_trials=200)
var_form = RY(qubitOp.num_qubits, depth=args.vqe_depth, entanglement=args.vqe_entangler) 
algo = VQE(qubitOp, var_form, optimizer)
result = algo.run(quantum_instance)  
## save VQE results
filename_vqe = args.outpath_vqe \
             + args.molecule + '_' \
             + args.basis_set + '_' \
             + d_string + '_VQE_SPSA_RY_lin_bk_' \
             + str(args.num_shots) + '.pkl'
filehandler_vqe = open(filename_vqe, 'wb')
pickle.dump(result, filehandler_vqe)
print("saved file = ", filename_vqe)

/home/ubuntu/mol2qpu/output/VQE_results/H2_ccpvdz_0pt55_VQE_SPSA_RY_lin_bk_1000.pkl


In [32]:
print("VQE settings = ", algo.print_settings())
print("VQE result = ", result)

VQE settings =  
==================== Setting of VQE ============================
Algorithm: VQE
-- quantum_instance: 
Qiskit Terra version: 0.12.0
Backend: 'qasm_simulator (AerProvider)', with following setting:
{'basis_gates': ['u1', 'u2', 'u3', 'cx', 'cz', 'id', 'x', 'y', 'z', 'h', 's', 'sdg', 't', 'tdg', 'swap', 'ccx', 'unitary', 'initialize', 'cu1', 'cu2', 'cu3', 'cswap', 'mcx', 'mcy', 'mcz', 'mcu1', 'mcu2', 'mcu3', 'mcswap', 'multiplexer', 'kraus', 'roerror'], 'coupling_map': None}
{'pass_manager': None, 'initial_layout': None, 'seed_transpiler': None, 'optimization_level': None}
RunConfig(max_credits=10, shots=1024)
{'timeout': None}
{'backend_options': {'max_parallel_threads': 10, 'max_parallel_experiments': 0, 'shots': 10}}
{}
Measurement mitigation: None
-- var_form: <qiskit.aqua.components.variational_forms.ry.RY object at 0x7ff01378d9d0>
-- optimizer: <qiskit.aqua.components.optimizers.spsa.SPSA object at 0x7ff01378ddc0>
-- cost_fn: <bound method VQE._energy_evaluation of <

In [28]:
## load VQE results
#print(filename_vqe)
#result_load = pickle.load(open(filename_vqe,'rb'))
#result_load

/home/ubuntu/mol2qpu/output/VQE_results/H2_ccpvdz_0pt55_VQE_SPSA_RY_lin_bk.pkl


{'num_optimizer_evals': None,
 'min_val': 24.434041564524765,
 'opt_params': array([ 1.27838137,  0.93962972, -1.21642166,  1.99249258, -3.36752337,
         0.22872054,  2.91283336, -2.05564601, -1.48489161, -0.90719709,
         0.05035043, -0.8611353 ,  1.38032328,  0.72537749,  1.85646108,
         1.23967499,  1.33578385,  0.94867126, -0.0303317 ,  3.48702445,
         0.15098988, -1.49079323,  0.17772595, -0.11110189, -2.89392263,
         1.18214993, -0.25386109, -1.13967618, -0.93000068,  1.15689503,
         1.3611201 , -4.27124984,  1.35956809,  2.35701586, -3.8871554 ,
        -5.38495828, -1.16899272, -3.20994809,  1.28760584, -0.47160713]),
 'eval_time': 25632.190756559372,
 'eval_count': 451,
 'energy': 24.434041564524765,
 'eigvals': array([24.43404156]),
 'min_vector': {'10100111110011001000': 1,
  '11010101110011001110': 1,
  '10100011111111111101': 1,
  '10100101000001001000': 1,
  '10111011111111101000': 1,
  '10001111010011000000': 1,
  '10111110000001101011': 1,
  